In [ ]:
# import the necessary packages
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from scipy import ndimage
from google.colab.patches import cv2_imshow
import imutils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def black_and_white_rounder(x, base=255):
  if x > 50:
    return 1
  return 0 
  # result =  base * round(float(x) / base)
black_and_white_rounder(128)  

1

In [ ]:
def remove_small_areas(matrix3):
  img = (matrix3 * 1).astype("uint8")
  gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
  # 
  #find all your connected components (white blobs in your image)
  nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)
  #connectedComponentswithStats yields every seperated component with information on each of them, such as size
  #the following part is just taking out the background which is also considered a component, but most of the time we don't want that.
  sizes = stats[1:, -1]; nb_components = nb_components - 1
  print(max(sizes)/nb_components)
  # minimum size of particles we want to keep (number of pixels)
  #here, it's a fixed value, but you can set it as you want, eg the mean of the sizes or whatever
  min_size = 15000 

  #your answer image
  img2 = np.zeros((output.shape),np.uint8)
  #for every component in the image, you keep it only if it's above min_size
  for i in range(0, nb_components):
      if sizes[i] >= min_size:
          img2[output == i + 1] = 255
  # cv2_imshow(img2)
  matrix4 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
  return matrix4

In [ ]:
dataset_path = "/content/drive/MyDrive/Database/ARC DATABASE/dataset_info_by_date.csv"
df = pd.read_csv(dataset_path)

for index, row in df.iterrows():
    print(row)
    try:
      file_path = row['full_path']
      image = cv2.imread(file_path)
      saliency = cv2.saliency.StaticSaliencySpectralResidual_create()
      (success, saliencyMap) = saliency.computeSaliency(image)
      saliencyMap = (saliencyMap * 255).astype("uint8")

      img = Image.fromarray(saliencyMap)
      new_path = file_path.replace(".jpg", "saliency_map.png")
      img.save(new_path) # Image saving to another directory
      
      # 
      file_path = row['full_path'].replace(".jpg", "saliency_map.png")
      matrix1 = cv2.imread(file_path)
      matrix2 = cv2.imread(row['full_path'])
      # 
      squarer = lambda t: black_and_white_rounder(t)
      vfunc = np.vectorize(squarer)
      matrix4 = vfunc(matrix1)

      matrix3 = np.multiply(matrix4,matrix2)
      # 
      matrix4 = remove_small_areas(matrix3)
      # 

      map = np.argwhere( (matrix4[:, :, 0:3] != [0,0,0]).all(2))
      min_y = min(map[:,0])
      min_x = min(map[:,1])
      max_y = max(map[:,0])
      max_x = max(map[:,1])
      print(min_y,min_x,max_y,max_x)
      cropped = matrix2[min_y:max_y,min_x:max_x]

      # 
      # cropped_array = (cropped * 255).astype("uint8")
      # img = Image.fromarray(cropped_array)
      new_path = row['full_path'].replace(".jpg", "_cropped.png")
      # img.save(new_path) # Image saving to another directory
      cv2.imwrite(new_path,cropped)
      print(new_path)

    except Exception as err:
      print(err)
      continue

id                                                  2778569169971553819
username                                                       le_blanc
full_path             /content/drive/MyDrive/Database/ARC DATABASE/l...
account_created_at                                  2011-08-14 00:00:00
posted_at                                                    2022-02-21
days_between                                                       3844
likes                                                       2260.662043
like_rate                                                      0.009483
Name: 0, dtype: object
5504.285714285715
712 0 1439 1439
/content/drive/MyDrive/Database/ARC DATABASE/le_blanc/2022-02-21_13-47-58_UTC_cropped.png
id                                                  2774931022469536859
username                                                       le_blanc
full_path             /content/drive/MyDrive/Database/ARC DATABASE/l...
account_created_at                                  2011-08-1